In [1]:
import numpy as np
import pandas as pd

In [9]:
path = '/content/drive/MyDrive/통계데이터센터/'

train = pd.read_csv( path + 'train_preprocessing_aug.csv',encoding='cp949')
test = pd.read_csv( path + 'test_preprocessing.csv',encoding='cp949')
submission = pd.read_csv(path + 'submission.csv')

In [10]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
train['digit_3'] = encoder.fit_transform(train['digit_3'].values)

In [11]:
display(encoder.classes_)

array([ 11,  12,  14,  20,  31,  32,  51,  61,  62,  71,  72,  80, 101,
       102, 103, 104, 105, 106, 107, 108, 111, 112, 120, 131, 132, 133,
       134, 139, 141, 142, 143, 144, 151, 152, 161, 162, 163, 171, 172,
       179, 181, 182, 191, 192, 201, 202, 203, 204, 205, 211, 212, 213,
       221, 222, 231, 232, 233, 239, 241, 242, 243, 251, 252, 259, 261,
       262, 263, 264, 265, 266, 271, 272, 273, 274, 281, 282, 283, 284,
       285, 289, 291, 292, 301, 302, 303, 304, 311, 312, 313, 319, 320,
       331, 332, 333, 334, 339, 340, 351, 352, 353, 360, 370, 381, 382,
       383, 390, 411, 412, 421, 422, 423, 424, 425, 426, 451, 452, 453,
       461, 462, 463, 464, 465, 466, 467, 468, 471, 472, 473, 474, 475,
       476, 477, 478, 479, 491, 492, 493, 494, 495, 501, 502, 511, 512,
       521, 529, 551, 559, 561, 562, 581, 582, 591, 592, 601, 602, 611,
       612, 620, 631, 639, 641, 642, 649, 651, 652, 653, 661, 662, 681,
       682, 701, 702, 711, 712, 713, 714, 715, 716, 721, 729, 73

In [13]:
Roberta30 = np.load(path + 'proba/Roberta30.npy')
Roberta50 = np.load(path + 'proba/Roberta50.npy')
Roberta70 = np.load(path + 'proba/Roberta70.npy')
Roberta120 = np.load(path + 'proba/Roberta120.npy')
kobert = np.load(path + 'proba/kobert_dp.npy')

In [17]:
ensemble = ((Roberta30 + Roberta50 + Roberta70 + Roberta120)/4 + kobert)/2

In [23]:
d3 = []

for i in ensemble:
  a = np.argmax(i)
  d3.append(a)

In [24]:
encoder.inverse_transform(d3)

array([561, 466, 949, ..., 478, 902, 682], dtype=int64)

In [25]:
submission['digit_3'] = encoder.inverse_transform(d3)

In [26]:
def finddigit2(k):
    digit2 = k.copy()
    digit2['digit_3']= digit2['digit_3'].apply(lambda x : int(x))
    digit2['digit_2']= digit2['digit_3'].apply(lambda x : str(x))
    digit2['digit_2'] = digit2['digit_2'].apply(lambda x : int(x[:2]) if len(x)==3 else int(x[0]))
    
    return digit2

In [27]:
def industry(num):
    if num >= 1 and num <= 3:
        result  = 'A'
    elif num >= 5 and num <= 8:
        result  = 'B'
    elif num >= 10 and num <= 34:
        result  = 'C'
    elif num ==35:
        result  = 'D'        
    elif num >= 36 and num <= 39:
        result  = 'E'
    elif num >= 41 and num <= 42:
        result  = 'F'
    elif num >= 45 and num <= 47:
        result  = 'G'
    elif num >= 49 and num <= 52:
        result  = 'H'
    elif num >= 55 and num <= 56:
        result  = 'I'
    elif num >= 58 and num <= 63:
        result  = 'J'
    elif num >= 64 and num <= 66:
        result  = 'K'
    elif num == 68:
        result  = 'L'
    elif num >= 70 and num <= 73:
        result  = 'M'
    elif num >= 74 and num <= 76:
        result  = 'N'
    elif num == 84:
        result  = 'O'
    elif num == 85:
        result  = 'P'
    elif num >= 86 and num <= 87:
        result  = 'Q'
    elif num >= 90 and num <= 91:
        result  = 'R'
    elif num >= 94 and num <= 96:
        result  = 'S'
    elif num >= 97 and num <= 98:
        result  = 'T'
    elif num ==99:
        result  = 'U'
    return result

In [30]:
submission = finddigit2(submission)
submission['digit_1'] = submission['digit_2'].apply(lambda x: industry(x))

In [32]:
submission

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,I,56,561,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,G,46,466,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,S,94,949,절에서,신도을 대상으로,불교단체운영
3,id_000004,S,95,952,영업장에서,고객요구로,자동차튜닝
4,id_000005,I,56,562,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,A,1,11,사업장에서,일반인대상으로,버섯농장
99996,id_099997,Q,86,862,한의원에서,외래환자위주고,치료
99997,id_099998,G,47,478,일반점포에서,소비자에게,그림판매
99998,id_099999,R,90,902,사업장에서,일반인.학생대상으로,학습공간제공


In [ ]:
submission.to_csv(path + '산업분류예측값.csv', index=False, encoding='cp949')